In [1]:
dataset = [[1,'High','Good','Pass'],
           [2,'High','Poor','Pass'],
           [3,'Low','Good','Pass'],
           [4,'Low','Poor','Fail]'],
           [5,'High','Good','Pass'],
           [6,'Low','Poor','Fail']
          ]

In [78]:
dataset = [[1,'High','Good','Pass'],
           [2,'High','Poor','Pass'],
           [3,'Low','Good','Pass'],
           [4,'Low','Poor','Fail'], 
           [5,'High','Good','Pass'],
           [6,'Low','Poor','Fail']
          ]

def PreProcess_Data(dataset):
    data = [row[:] for row in dataset]  
    for i in range(len(data)):
        # Encode High=1, Low=0
        if data[i][1] == 'High':
            data[i][1] = 1
        elif data[i][1] == 'Low':
            data[i][1] = 0

        # Encode Good=1, Poor=0
        if data[i][2] == 'Good':
            data[i][2] = 1
        elif data[i][2] == 'Poor':
            data[i][2] = 0

        # Encode Pass=1, Fail=0
        if data[i][3] == 'Pass':
            data[i][3] = 1
        elif data[i][3] == 'Fail':
            data[i][3] = 0
    
    # print("Original Data: ", dataset)
    print("PreProcessed Data: ", data)
    return data

data = PreProcess_Data(dataset)

PreProcessed Data:  [[1, 1, 1, 1], [2, 1, 0, 1], [3, 0, 1, 1], [4, 0, 0, 0], [5, 1, 1, 1], [6, 0, 0, 0]]


In [ ]:
# Apply the Bayes classifier to the Student Performance dataset without using any model library
# (implement from counts). Compute the class priors and likelihoods from the training data, use
# them to calculate the posterior probabilities for the new record (Study = High, Attendance =
# Poor), and report the predicted class (Pass or Fail) together with the intermediate calculations.

In [25]:
import numpy as np

def priorProb(data):
    classCount={}
    for row in data:
        label=row[-1]
        if label not in classCount:
            classCount[label]=0
        classCount[label]+=1
    total=len(data)
    priors={}
    for label,count in classCount.items():
        priors[label]=count/total
    return priors
priorProbability=print(priorProb(data))
priorProbability




{1: 0.6666666666666666, 0: 0.3333333333333333}


In [20]:
highPass=0
highFail=0

lowPass=0
lowFail=0


for row in data:
    if row[1]==1 and row[3]==1:
        highPass+=1
    elif row[1]==1 and row[3]==0:
        highFail+=1
    elif row[1]==0 and row[3]==1:
        lowPass+=1
    elif row[1]==0 and row[3]==0:
        lowFail+=1
print("High Pass: ", highPass/len(data))
print("High Fail: ", highFail/len(data))
print("Low Pass: ", lowPass/len(data))
print("Low Fail: ", lowFail/len(data))

High Pass:  0.5
High Fail:  0.0
Low Pass:  0.16666666666666666
Low Fail:  0.3333333333333333


In [32]:

def calculate_likelihoods(data):
    pass_data = [row for row in data if row[3] == 1]  
    fail_data = [row for row in data if row[3] == 0]  
    

    print("pass samples: ", len(pass_data))
    print("fail samples: ", len(fail_data))

    totalPass = len(pass_data)
    totalFail= len(fail_data)

    study_high_given_pass = sum(1 for row in pass_data if row[1] == 1) / totalPass
    study_low_given_pass = sum(1 for row in pass_data if row[1] == 0) / totalPass   
    study_high_given_fail = sum(1 for row in fail_data if row[1] == 1) / totalFail
    study_low_given_fail = sum(1 for row in fail_data if row[1] == 0) / totalFail
    attend_good_given_pass = sum(1 for row in pass_data if row[2] == 1) / totalPass
    attend_poor_given_pass = sum(1 for row in pass_data if row[2] == 0) / totalPass
    attend_good_given_fail = sum(1 for row in fail_data if row[2] == 1) / totalFail
    attend_poor_given_fail = sum(1 for row in fail_data if row[2] == 0) / totalFail
     
    return {
        'study_high_pass': study_high_given_pass,
        'study_low_pass': study_low_given_pass,
        'study_high_fail': study_high_given_fail,
        'study_low_fail': study_low_given_fail,
        'attend_good_pass': attend_good_given_pass,
        'attend_poor_pass': attend_poor_given_pass,
        'attend_good_fail': attend_good_given_fail,
        'attend_poor_fail': attend_poor_given_fail
    }

likelihoods = calculate_likelihoods(data)

pass samples:  4
fail samples:  2


## QUESTION NUMBER 1

In [1]:
from collections import Counter, defaultdict

dataset = [[1,'High','Good','Pass'],
           [2,'High','Poor','Pass'],
           [3,'Low','Good','Pass'],
           [4,'Low','Poor','Fail'],
           [5,'High','Good','Pass'],
           [6,'Low','Poor','Fail']
          ]

data = [(row[1], row[2], row[3]) for row in dataset]
print("Data (Study, Attendance, Result):", data)

Data (Study, Attendance, Result): [('High', 'Good', 'Pass'), ('High', 'Poor', 'Pass'), ('Low', 'Good', 'Pass'), ('Low', 'Poor', 'Fail'), ('High', 'Good', 'Pass'), ('Low', 'Poor', 'Fail')]


In [ ]:
import pandas as pd
df=pd.DataFrame(data, columns=['Study', 'Attendance', 'Result'])
print("\nDataFrame:\n", df)


DataFrame:
   Study Attendance Result
0  High       Good   Pass
1  High       Poor   Pass
2   Low       Good   Pass
3   Low       Poor   Fail
4  High       Good   Pass
5   Low       Poor   Fail


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Study'] = le.fit_transform(df['Study'])
df['Attendance'] = le.fit_transform(df['Attendance'])
df['Result'] = le.fit_transform(df['Result'])
print("\nEncoded DataFrame:\n")
print(df.head(6),"\n\n")


#Bayes Classifier from scratch
import numpy as np
priorProb={}
for c in np.unique(df['Result']):
    priorProb[c]=len(df[df['Result']==c])/len(df)
print("Prior Probabilities:", priorProb)

# Likelihood Calculation
likelihoodProb={}
for c in np.unique(df['Result']):
    subset=df[df['Result']==c]
    likelihoodProb[c]={}
    for feature in ['Study', 'Attendance']:
        likelihoodProb[c][feature]={}
        for value in np.unique(df[feature]):
            likelihoodProb[c][feature][value]=len(subset[subset[feature]==value])/len(subset)
print("Likelihood P(Study|Pass):", list(likelihoodProb[1]['Study'].values()))
print("Likelihood P(Study|Fail):", list(likelihoodProb[0]['Study'].values()))
print("Likelihood P(Attendance|Pass):", list(likelihoodProb[1]['Attendance'].values()))
print("Likelihood P(Attendance|Fail):", list(likelihoodProb[0]['Attendance'].values()))

# Prediction for Study=High(1), Attendance=Poor(0)
test_instance={'Study':1, 'Attendance':0}
posteriorProb={}
for c in np.unique(df['Result']):
    posteriorProb[c]=priorProb[c]
    for feature, value in test_instance.items():
        posteriorProb[c]*=likelihoodProb[c][feature][value]
print("\nPosterior Probabilities:", list(posteriorProb.values()))
predicted_class=max(posteriorProb, key=posteriorProb.get)
print("\nPredicted Class (0=Fail, 1=Pass):", predicted_class)



Encoded DataFrame:

   Study  Attendance  Result
0      0           0       1
1      0           1       1
2      1           0       1
3      1           1       0
4      0           0       1
5      1           1       0 


Prior Probabilities: {np.int64(0): 0.3333333333333333, np.int64(1): 0.6666666666666666}
Likelihood P(Study|Pass): [0.75, 0.25]
Likelihood P(Study|Fail): [0.0, 1.0]
Likelihood P(Attendance|Pass): [0.75, 0.25]
Likelihood P(Attendance|Fail): [0.0, 1.0]

Posterior Probabilities: [0.0, 0.125]

Predicted Class (0=Fail, 1=Pass): 1


In [15]:
dataset = [[1,'High','Good','Pass'],
           [2,'High','Poor','Pass'],
           [3,'Low','Good','Pass'],
           [4,'Low','Poor','Fail'], 
           [5,'High','Good','Pass'],
           [6,'Low','Poor','Fail']
          ]

def PreProcess_Data(dataset):
    data = [row[:] for row in dataset]  
    for i in range(len(data)):
        # Encode High=1, Low=0
        if data[i][1] == 'High':
            data[i][1] = 1
        elif data[i][1] == 'Low':
            data[i][1] = 0

        # Encode Good=1, Poor=0
        if data[i][2] == 'Good':
            data[i][2] = 1
        elif data[i][2] == 'Poor':
            data[i][2] = 0

        # Encode Pass=1, Fail=0
        if data[i][3] == 'Pass':
            data[i][3] = 1
        elif data[i][3] == 'Fail':
            data[i][3] = 0
    
    # print("Original Data: ", dataset)
    print("PreProcessed Data: ", data)
    return data

data = PreProcess_Data(dataset)

PreProcessed Data:  [[1, 1, 1, 1], [2, 1, 0, 1], [3, 0, 1, 1], [4, 0, 0, 0], [5, 1, 1, 1], [6, 0, 0, 0]]


In [16]:
# Step 1. Priors P(C):
N = len(data)
class_counts = Counter([row[2] for row in data]) # row 2 for labels pass/fail
# print(class_counts)

priors = {c: class_counts[c] / N for c in class_counts} # Calculate priors by dividing class count by total samples

print("\nPriors:")
print(class_counts)
print("Priors:", priors)
print("pass prob ", priors[1])
print("fail prob ", priors[0])
print(class_counts[0])



Priors:
Counter({1: 3, 0: 3})
Priors: {1: 0.5, 0: 0.5}
pass prob  0.5
fail prob  0.5
3


In [58]:
# Step 2. Likelihoods P(X∣C) 
# Group data by class
features_given_class = defaultdict(list)
for study, attendance, label in data:
    features_given_class[label].append((study, attendance))

print("\nStep 2 - Feature groups by class:")
for label in features_given_class:
    print(f"{label}: {features_given_class[label]}")

# Likelihood function: P(feature=value | class)
def likelihood_study(study, c):
    values = [s for (s, a) in features_given_class[c]]
    return values.count(study) / len(values)

def likelihood_attendance(attendance, c):
    values = [a for (s, a) in features_given_class[c]]
    return values.count(attendance) / len(values)




Step 2 - Feature groups by class:
Pass: [('High', 'Good'), ('High', 'Poor'), ('Low', 'Good'), ('High', 'Good')]
Fail: [('Low', 'Poor'), ('Low', 'Poor')]


In [60]:
print("\nLikelihoods:")
for c in class_counts:
    print(f"\nFor class {c}:")
    print(f"  P(Study=High|{c}) = {likelihood_study('High', c)}")
    print(f"  P(Study=Low|{c}) = {likelihood_study('Low', c)}")
    print(f"  P(Attendance=Good|{c}) = {likelihood_attendance('Good', c)}")
    print(f"  P(Attendance=Poor|{c}) = {likelihood_attendance('Poor', c)}")

# Test case: Study = High, Attendance = Poor
study = 'High'
attendance = 'Poor'


Likelihoods:

For class Pass:
  P(Study=High|Pass) = 0.75
  P(Study=Low|Pass) = 0.25
  P(Attendance=Good|Pass) = 0.75
  P(Attendance=Poor|Pass) = 0.25

For class Fail:
  P(Study=High|Fail) = 0.0
  P(Study=Low|Fail) = 1.0
  P(Attendance=Good|Fail) = 0.0
  P(Attendance=Poor|Fail) = 1.0


In [66]:
# Step 3. Evidence P(X)
print("\nStep 3 - Calculate Evidence:")
numerators = {}
for c in class_counts:
    prior = priors[c]
    ls = likelihood_study(study, c)
    la = likelihood_attendance(attendance, c)
    likelihood = ls * la
    numerators[c] = likelihood * prior

evidence = sum(numerators.values())
print(f"\nTotal Evidence P(X) = {evidence}")

# multiplies likelihoods with priors to get numerators for each class



Step 3 - Calculate Evidence:

Total Evidence P(X) = 0.125


In [64]:
# Step 4. Posterior & Prediction
print("\nStep 4 - Posterior Probabilities:")
posteriors = {c: numerators[c]/evidence for c in numerators}
for c in posteriors:
    print(f"P({c}|X) = {numerators[c]:.6f} / {evidence:.6f} = {posteriors[c]:.6f}")

prediction = max(posteriors, key=posteriors.get)
confidence = posteriors[prediction]

print(f"\nFinal Prediction: {prediction}")
print(f"Confidence: {confidence:.6f} ({confidence*100:.2f}%)")


Step 4 - Posterior Probabilities:
P(Pass|X) = 0.125000 / 0.125000 = 1.000000
P(Fail|X) = 0.000000 / 0.125000 = 0.000000

Final Prediction: Pass
Confidence: 1.000000 (100.00%)


In [ ]:
# BAYES CLASS:
# Student Performance Dataset
dataset = [
    [1, 'High', 'Good', 'Pass'],
    [2, 'High', 'Poor', 'Pass'],
    [3, 'Low',  'Good', 'Pass'],
    [4, 'Low',  'Poor', 'Fail'],
    [5, 'High', 'Good', 'Pass'],
    [6, 'Low',  'Poor', 'Fail']
]

# Step 1: Separate data by class
classes = {}
for row in dataset:
    label = row[3]
    if label not in classes:
        classes[label] = []
    classes[label].append(row)
print("Classes:", classes)
# Step 2: Calculate class priors
total = len(dataset)
priors = {c: len(rows)/total for c, rows in classes.items()}

# Step 3: Calculate likelihoods for each feature given class
# def likelihood(feature_index, feature_value, class_label):
#     class_rows = classes[class_label]
#     count = sum(1 for row in class_rows if row[feature_index] == feature_value)
#     return count / len(class_rows)

# Step 3: Likelihood with Laplace smoothing
def likelihood(feature_index, feature_value, class_label):
   class_rows = classes[class_label]
   count = sum(1 for row in class_rows if row[feature_index] == feature_value)
   k = len(feature_value[feature_index])     # number of unique possible values for this feature
   return (count + 1) / (len(class_rows) + k)


# Step 4: Compute posteriors for new data
new_record = ['?', 'High', 'Poor']  # (Study=High, Attendance=Poor)

posteriors = {}
for c in classes:
    p_class = priors[c]
    p_features = 1

    # multiply likelihoods for all features (except ID and class label)
    for i in range(1, len(new_record)):
        p_features *= likelihood(i, new_record[i], c)

    posteriors[c] = p_class * p_features

# Step 5: Normalize (optional)
total_prob = sum(posteriors.values())
for c in posteriors:
    posteriors[c] = posteriors[c] / total_prob if total_prob != 0 else 0

# Step 6: Display intermediate results
print("Class Priors:", priors)
print("Posteriors (unnormalized):", posteriors)
print("Predicted Class:", max(posteriors, key=posteriors.get))


Classes: {'Pass': [[1, 'High', 'Good', 'Pass'], [2, 'High', 'Poor', 'Pass'], [3, 'Low', 'Good', 'Pass'], [5, 'High', 'Good', 'Pass']], 'Fail': [[4, 'Low', 'Poor', 'Fail'], [6, 'Low', 'Poor', 'Fail']]}
Class Priors: {'Pass': 0.6666666666666666, 'Fail': 0.3333333333333333}
Posteriors (unnormalized): {'Pass': 0.6575342465753425, 'Fail': 0.34246575342465746}
Predicted Class: Pass


## QUESTION NUMBMER 2

In [28]:

train = [ ("math exam tomorrow", "study"), ("submit assignment today", "study"),("play football now", "fun"),("go watch movie", "fun")]
test = ["study math", "watch football" ]

In [30]:
from collections import Counter
def tokenize(text):     # split text into words
    return text.lower().split()

# Count documents per class
class_counts = Counter([label for _, label in train])
total_docs = len(train)

# Priors
priors = {c: class_counts[c] / total_docs for c in class_counts}

# Word counts per class
word_counts = {c: Counter() for c in class_counts}
total_words = {c: 0 for c in class_counts}

for text, label in train:
    words = tokenize(text)
    for w in words:
        word_counts[label][w] += 1
        total_words[label] += 1

# Step 4: Prediction
def predict(text):
    words = tokenize(text)
    scores = {}
    for c in class_counts:
        # Start with prior
        score = priors[c]
        # print (score)
        for w in words:
            if word_counts[c][w] > 0:
                score *= word_counts[c][w] / total_words[c]
            else:
                score *= 0  # unseen word => probability 0
        scores[c] = score
    # Pick class with max score
    pred = max(scores, key=scores.get)
    return pred, scores

# Step 5: Test
for t in test:
    pred, scores = predict(t)
    print("Text:", t, "→ Predicted:", pred, "Scores:", scores)


# P(w|c) = (count(w,c) + 1) / (total words in c + V) added 1 for Laplace smoothing because of unseen words in test data

# Step 4 (with Laplace smoothing)
def predict_laplace(text):
    words = tokenize(text)
    scores = {}
    V = len(set(w for text, _ in train for w in tokenize(text)))  # vocabulary size
    for c in class_counts:
        # Start with prior
        score = priors[c]
        for w in words:
            # Apply Laplace smoothing
            score *= (word_counts[c][w] + 1) / (total_words[c] + V)
        scores[c] = score
    pred = max(scores, key=scores.get)
    return pred, scores

# Step 5: Test again (with Laplace)
print("\n--- With Laplace Smoothing ---")
for t in test:
    pred, scores = predict_laplace(t)
    print("Text:", t, "→ Predicted:", pred, "Scores:", scores)

Text: study math → Predicted: study Scores: {'study': 0.0, 'fun': 0.0}
Text: watch football → Predicted: fun Scores: {'study': 0.0, 'fun': 0.013888888888888888}

--- With Laplace Smoothing ---
Text: study math → Predicted: study Scores: {'study': 0.0030864197530864196, 'fun': 0.0015432098765432098}
Text: watch football → Predicted: fun Scores: {'study': 0.0015432098765432098, 'fun': 0.006172839506172839}


In [ ]:
# Prior: P(study) = 0.5
# P("study"|study) = 1/6 = 0.167 (word "study" appears once in 6 total study words)
# P("math"|study) = 1/6 = 0.167 (word "math" appears once in 6 total study words)
# Score = 0.5 × 0.167 × 0.167 = 0.014

# Prior: P(fun) = 0.5
# P("study"|fun) = 0/6 = 0.0 (word "study" never appears in fun class)
# P("math"|fun) = 0/6 = 0.0 (word "math" never appears in fun class)
# Score = 0.5 × 0.0 × 0.0 = 0.0

## QUESTION NUMBER 3

In [72]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

# Dataset
X = np.array([
    [85, 78],
    [72, 65],
    [50, 45],
    [60, 52],
    [90, 88]
])

y = np.array(["Pass", "Pass", "Fail", "Fail", "Pass"])

In [ ]:
# Train Gaussian Naive Bayes model
model = GaussianNB()
model.fit(X, y)

,priors,None
,var_smoothing,1e-09


In [ ]:
test_samples = np.array([[80, 70], [55, 50]])
predicted_classes = model.predict(test_samples)
predicted_probs = model.predict_proba(test_samples)

In [ ]:
# Show results
for i, sample in enumerate(test_samples):
    print(f"Student marks: {sample}")
    print(f" Predicted Class: {predicted_classes[i]}")
    print(f" Class Probabilities: {predicted_probs[i]}\n")

Student marks: [80 70]
 Predicted Class: Pass
 Class Probabilities: [8.96752131e-14 1.00000000e+00]

Student marks: [55 50]
 Predicted Class: Fail
 Class Probabilities: [9.99989976e-01 1.00235389e-05]

